In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# Chargement des données

In [ ]:
df_non_weather_features = pd.read_csv('/content/drive/MyDrive/ENPC/2A/HACKATHON 2/waiting_times_train.csv')
df_non_weather_features['DATETIME'] = pd.to_datetime(df_non_weather_features['DATETIME'])
df_non_weather_features = pd.get_dummies(df_non_weather_features, columns=['ENTITY_DESCRIPTION_SHORT'])

true_labels = df_non_weather_features['WAIT_TIME_IN_2H']
df_non_weather_features = df_non_weather_features.drop('WAIT_TIME_IN_2H', axis=1)

df_weather_features = pd.read_csv('/content/drive/MyDrive/ENPC/2A/HACKATHON 2/weather_data.csv')
df_weather_features['DATETIME'] = pd.to_datetime(df_weather_features['DATETIME'])

df = pd.merge(df_non_weather_features, df_weather_features, on='DATETIME')

In [ ]:
def map_month_to_season(month_number):
  if 3 <= month_number <= 5:
    return "spring"
  elif 6 <= month_number <= 8:
    return "summer"
  elif 9 <= month_number <= 11:
    return "autumn"
  else:
    return "winter"


def map_hour_to_moment(hour):
  """moment = period of the day (morning, lunch or afternoon)"""
  if hour <= 11:
    return "morning"
  elif 12 <= hour <= 13:
    return "lunch"
  elif 14 <= hour <= 16:
    return "afternoon"
  else:
    return "evening"


df['weekend'] = (df['DATETIME'].dt.weekday > 4).astype(int)

df['season'] = df['DATETIME'].map(lambda x: map_month_to_season(x.month))
df = pd.get_dummies(df, columns=['season'])

df['moment'] = df['DATETIME'].map(lambda x: map_hour_to_moment(x.hour))
df = pd.get_dummies(df, columns=['moment'])

df = df.drop('DATETIME', axis=1)

In [ ]:
"""Ces donnees ont ete calculees sur l'ensemble d'entrainement (il est important
de ne pas les recalculer lors de l'evaluation sur les set de validation et de
test)"""

continuous_columns_min = {
    'ADJUST_CAPACITY': 3.400000,
    'TIME_TO_PARADE_1': -480.000000,
    'TIME_TO_PARADE_2': -595.000000,
    'TIME_TO_NIGHT_SHOW': -165.000000,
    'WAIT_TIME_IN_2H': 0.000000,
    'temp': -2.732500,
    'dew_point': -7.320000,
    'feels_like': -6.670000,
    'pressure': 982.000000,
    'humidity': 17.000000,
    'wind_speed': 0.060000,
    'rain_1h': 0.100000,
    'snow_1h': 0.110216,
    'clouds_all': 0.000000
}

continuous_columns_max = {
    'ADJUST_CAPACITY': 756.000000,
    'TIME_TO_PARADE_1': 570.000000,
    'TIME_TO_PARADE_2': 235.000000,
    'TIME_TO_NIGHT_SHOW': 840.000000,
    'WAIT_TIME_IN_2H': 155.000000,
    'temp': 41.260000,
    'dew_point': 21.802500,
    'feels_like': 40.850000,
    'pressure': 1047.000000,
    'humidity': 100.000000,
    'wind_speed': 12.180000,
    'rain_1h': 7.145455,
    'snow_1h': 1.598148,
    'clouds_all': 100.000000
}

non_binary_int_columns_min = {
    'CURRENT_WAIT_TIME': 0,
    'DOWNTIME': 0
}

non_binary_int_columns_max = {
    'CURRENT_WAIT_TIME': 155,
    'DOWNTIME': 14
}

In [ ]:
# On normalise les variables non one hot avec une transformation affine pour les ramener dans [0, 1]
continuous_columns = df.select_dtypes(include=['float64'])
non_binary_int_columns = ['CURRENT_WAIT_TIME', 'DOWNTIME']

for column_name in continuous_columns.columns:
  df[column_name] = (df[column_name] - continuous_columns_min[column_name]) / (continuous_columns_max[column_name] - continuous_columns_min[column_name])
for column_name in non_binary_int_columns:
  df[column_name] = (df[column_name] - non_binary_int_columns_min[column_name]) / (non_binary_int_columns_max[column_name] - non_binary_int_columns_min[column_name])

df.head()

,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,ENTITY_DESCRIPTION_SHORT_Flying Coaster,ENTITY_DESCRIPTION_SHORT_Pirate Ship,ENTITY_DESCRIPTION_SHORT_Water Ride,temp,...,clouds_all,weekend,season_autumn,season_spring,season_summer,season_winter,moment_afternoon,moment_evening,moment_lunch,moment_morning
0,0.323678,0.0,0.129032,NaN,NaN,NaN,0,0,1,0.198898,...,0.64,1,0,0,0,1,0,0,0,1
1,1.000000,0.0,0.322581,NaN,NaN,NaN,1,0,0,0.198898,...,0.64,1,0,0,0,1,0,0,0,1
2,0.198778,0.0,0.322581,NaN,NaN,NaN,0,1,0,0.198898,...,0.64,1,0,0,0,1,0,0,0,1
3,0.323678,0.0,0.193548,0.814286,NaN,0.656716,0,0,1,0.242144,...,0.00,1,0,0,0,1,0,0,0,1
4,1.000000,0.0,0.096774,0.814286,NaN,0.656716,1,0,0,0.242144,...,0.00,1,0,0,0,1,0,0,0,1


Ensuite il faut supprimer les features inutiles et mettre des valeurs à la place des NaN

In [ ]:
df = df.dropna(axis=1)

df.head()

,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,ENTITY_DESCRIPTION_SHORT_Flying Coaster,ENTITY_DESCRIPTION_SHORT_Pirate Ship,ENTITY_DESCRIPTION_SHORT_Water Ride,temp,dew_point,feels_like,pressure,...,clouds_all,weekend,season_autumn,season_spring,season_summer,season_winter,moment_afternoon,moment_evening,moment_lunch,moment_morning
0,0.323678,0.0,0.129032,0,0,1,0.198898,0.422096,0.216751,0.688462,...,0.64,1,0,0,0,1,0,0,0,1
1,1.000000,0.0,0.322581,1,0,0,0.198898,0.422096,0.216751,0.688462,...,0.64,1,0,0,0,1,0,0,0,1
2,0.198778,0.0,0.322581,0,1,0,0.198898,0.422096,0.216751,0.688462,...,0.64,1,0,0,0,1,0,0,0,1
3,0.323678,0.0,0.193548,0,0,1,0.242144,0.120354,0.266835,0.826923,...,0.00,1,0,0,0,1,0,0,0,1
4,1.000000,0.0,0.096774,1,0,0,0.242144,0.120354,0.266835,0.826923,...,0.00,1,0,0,0,1,0,0,0,1


# Entraînement

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

USE_GPU = True
dtype = torch.float32
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('cpu')

In [ ]:
# Préparation des données
X = df.values
y = true_labels.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=dtype, device=device)
y_train = torch.tensor(y_train, dtype=dtype, device=device)
X_test = torch.tensor(X_test, dtype=dtype, device=device)
y_test = torch.tensor(y_test, dtype=dtype, device=device)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# Configuration du modèle en utilisant Sequential
model = nn.Sequential(
    nn.Linear(X.shape[1], 1000),
    nn.ReLU(),
    nn.Linear(1000, 1000),
    nn.ReLU(),
    nn.Linear(1000, 1)
).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Entraînement du modèle
num_epochs = 20
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 226.7752
Epoch [2/20], Loss: 241.2657


KeyboardInterrupt: 